## POS tagging using modified Viterbi

### Data Preparation

In [133]:
#Importing libraries
import nltk, re, pprint
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pprint, time
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [134]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

As mentioned in the Assignment splitting the data into train and test with 95:5 ratio

In [135]:
random.seed(1234)
training_set, validation_set = train_test_split(nltk_data,test_size=0.05)

print(len(training_set))
print(len(validation_set))

3718
196


Now,We get the List of Tagged Words

In [136]:
tagged_words = [tup for sent in training_set for tup in sent]

In [137]:
len(train_tagged_words)

95406

In [138]:
tagged_words[:10]

[('On', 'ADP'),
 ('Wall', 'NOUN'),
 ('Street', 'NOUN'),
 ('men', 'NOUN'),
 ('and', 'CONJ'),
 ('women', 'NOUN'),
 ('walk', 'VERB'),
 ('with', 'ADP'),
 ('great', 'ADJ'),
 ('purpose', 'NOUN')]

Extracting tokens from tagged words


In [139]:
tokens = [w[0] for w in tagged_words]

In [140]:
tokens[:10]

['On',
 'Wall',
 'Street',
 'men',
 'and',
 'women',
 'walk',
 'with',
 'great',
 'purpose']

Building vocaulary using tokens

In [141]:
voc = set(tokens)

In [142]:
print(len(voc))

12076


No of Tags

In [143]:
tags = set([w[1] for w in tagged_words])
print("No of Tags ", len(tags))

No of Tags  12


List of Tags

In [144]:
print(tags)

{'PRT', 'ADV', 'PRON', 'VERB', 'ADP', 'X', 'CONJ', 'DET', 'NOUN', 'NUM', '.', 'ADJ'}


### Build the vanilla Viterbi based POS tagger

In [145]:
# Calculation P(w/t) 
# Framing a matrix based on the calculated values
t = len(tags)
v = len(voc)
w_given_t = np.zeros((t, v))

In [146]:
# Emission Probability
# For a given tag compute the word
def word_from_tag(word, tag, bag = tagged_words):
    taglist = [p for p in bag if p[1]==tag]
    tagcount = len(taglist)
    w_given_tag_list = [p[0] for p in taglist if p[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, tagcount)

In [147]:
#Transition Probability
# Given tag1(t1) Compute tag2(t2)
def tag2_given_tag1(t2, t1, bag = tagged_words):
    tags = [p[1] for p in bag]
    t1_count = len([t for t in tags if t==t1])
    t2_t1_count = 0
    for i in range(len(tags)-1):
        if tags[i]==t1 and tags[i+1] == t2:
            t2_t1_count += 1
    return (t2_t1_count, t1_count)

Creating a Transition Matrix of tags with dimenstion t x t where each row is t1 and and each column is t2.

In [148]:
tag_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tag_matrix[i, j] = tag2_given_tag1(t2, t1)[0]/tag2_given_tag1(t2, t1)[1]

tag_matrix

array([[1.96270854e-03, 1.01406612e-02, 1.76643766e-02, 4.00719672e-01,
        1.99542027e-02, 1.37389600e-02, 2.28982652e-03, 1.02715082e-01,
        2.45011449e-01, 5.78999035e-02, 4.28524688e-02, 8.50507021e-02],
       [1.40140140e-02, 7.84117430e-02, 1.56823490e-02, 3.47347349e-01,
        1.19119123e-01, 2.26893555e-02, 7.00700702e-03, 7.04037398e-02,
        3.20320316e-02, 3.03636976e-02, 1.34467795e-01, 1.28461793e-01],
       [1.22840693e-02, 3.33973132e-02, 8.06142017e-03, 4.85604614e-01,
        2.22648755e-02, 9.36660245e-02, 5.37428027e-03, 9.59692895e-03,
        2.10748553e-01, 7.29366625e-03, 3.99232246e-02, 7.17850253e-02],
       [3.19627635e-02, 8.10705945e-02, 3.54538411e-02, 1.68968186e-01,
        9.09231976e-02, 2.18386352e-01, 5.43056615e-03, 1.35143518e-01,
        1.09775022e-01, 2.32738554e-02, 3.49883623e-02, 6.46237358e-02],
       [1.49796705e-03, 1.31607102e-02, 6.84784949e-02, 8.34581628e-03,
        1.65846348e-02, 3.49882320e-02, 7.48983526e-04, 3.25

Converting the tag matrix into a data frame

In [149]:
df_tags = pd.DataFrame(tag_matrix, columns = list(tags), index=list(tags))
df_tags

,PRT,ADV,PRON,VERB,ADP,X,CONJ,DET,NOUN,NUM,.,ADJ
PRT,0.001963,0.010141,0.017664,0.400720,0.019954,0.013739,0.002290,0.102715,0.245011,0.057900,0.042852,0.085051
ADV,0.014014,0.078412,0.015682,0.347347,0.119119,0.022689,0.007007,0.070404,0.032032,0.030364,0.134468,0.128462
PRON,0.012284,0.033397,0.008061,0.485605,0.022265,0.093666,0.005374,0.009597,0.210749,0.007294,0.039923,0.071785
VERB,0.031963,0.081071,0.035454,0.168968,0.090923,0.218386,0.005431,0.135144,0.109775,0.023274,0.034988,0.064624
ADP,0.001498,0.013161,0.068478,0.008346,0.016585,0.034988,0.000749,0.325915,0.322919,0.062808,0.039161,0.105393
X,0.183593,0.025547,0.055538,0.205966,0.144716,0.075056,0.010632,0.053475,0.062044,0.002856,0.163440,0.017137
CONJ,0.005104,0.055220,0.058005,0.156845,0.052900,0.008817,0.000464,0.118329,0.348956,0.041763,0.036659,0.116937
DET,0.000241,0.012535,0.003736,0.039412,0.009039,0.046161,0.000482,0.005303,0.636736,0.022659,0.018079,0.205616
NOUN,0.043744,0.017141,0.004731,0.145789,0.176214,0.029223,0.042543,0.013138,0.265121,0.009680,0.240847,0.011828
NUM,0.026971,0.002964,0.001482,0.018079,0.035566,0.208062,0.013634,0.003260,0.355365,0.183758,0.117664,0.033195


In [150]:
df_tags.loc['.', :]

PRT     0.002508
ADV     0.052137
PRON    0.066380
VERB    0.089761
ADP     0.091373
X       0.026606
CONJ    0.058318
DET     0.172355
NOUN    0.223865
NUM     0.079996
.       0.093165
ADJ     0.043447
Name: ., dtype: float32

### Plain vanilla Viterbi based POS tagger

In [151]:
# Viterbi Heuristic
def Viterbi(words, bag = tagged_words):
    state = []
    T = list(set([p[1] for p in bag]))
    
    for key, word in enumerate(words):
        #list initialisation for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = df_tags.loc['.', tag]
            else:
                transition_p = df_tags.loc[state[-1], tag]
                
            # emission and state probabilities
            emission_p = word_from_tag(words[key], tag)[0]/word_from_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # Finding the state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        print(key)
    return list(zip(words, state))

### Evaluating on Validation Set

In [152]:
#tagged words
validation_run_base = [tup for sent in validation_set for tup in sent]

In [153]:
#untagged words
validation_tagged_words = [tup[0] for sent in validation_set for tup in sent]

In [154]:
print(len(validation_set))
print(len(validation_tagged_words))

196
4972


Running Viterbi on validation set and checking the time taken

In [155]:
def cal_time(tag_words):
    start = time.time()
    V = Viterbi(tag_words)
    end = time.time()
    diff = end-start
    return V,diff

In [156]:
validation_Viterbi1,diff1 = cal_time(validation_tagged_words)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700


In [157]:
print("Time taken in seconds: ", diff1)
print("Number of words ",len(validation_tagged_words))

Time taken in seconds:  1988.7251210212708
Number of words  4972


In [158]:
#Accuracy
check = [i for i, j in zip(validation_Viterbi1, validation_run_base) if i == j] 
accuracy = len(check)/len(validation_Viterbi1)

print(accuracy)

0.9000402252614642


### Accuracy is 90%

Incorrect tagged cases in Vaidation Set

In [159]:
incorrect_tagged_cases = [[validation_run_base[i-1],j] for i, j in enumerate(zip(validation_Viterbi1, validation_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('.', '.'), (('So', 'ADV'), ('So', 'ADP'))],
 [('*-2', 'X'), (('successfully', 'PRT'), ('successfully', 'ADV'))],
 [('successfully', 'ADV'), (('mobilizing', 'PRT'), ('mobilizing', 'VERB'))],
 [('to', 'PRT'), (('bludgeon', 'PRT'), ('bludgeon', 'VERB'))],
 [('their', 'PRON'), (('tormentors', 'PRT'), ('tormentors', 'NOUN'))],
 [('the', 'DET'), (('public', 'ADJ'), ('public', 'NOUN'))],
 [('highly', 'ADV'), (('romanticized', 'PRT'), ('romanticized', 'VERB'))],
 [('no', 'DET'), (('more', 'ADJ'), ('more', 'ADV'))],
 [('more', 'ADV'), (('romanticized', 'PRT'), ('romanticized', 'VERB'))],
 [('the', 'DET'), (('good-hearted', 'PRT'), ('good-hearted', 'ADJ'))],
 [('.', '.'), (('Ten', 'PRT'), ('Ten', 'NUM'))],
 [('Ten', 'NUM'), (('shirt-sleeved', 'PRT'), ('shirt-sleeved', 'ADJ'))],
 [('a', 'DET'), (('circle', 'PRT'), ('circle', 'NOUN'))],
 [('one', 'NUM'), (('foot', 'PRT'), ('foot', 'NOUN'))],
 [('a', 'DET'), (('prize-fighter', 'PRT'), ('prize-fighter', 'NOUN'))],
 [('prize-fighter', 'NOUN'), (("

In [160]:
print(len(incorrect_tagged_cases))

497


#### Evaluatin using the Sample Sentences Provided

In [161]:
Test_sentences = pd.read_csv("Test_sentences.txt", sep = "\t", names=["message"])
Test_sentences

,message
0,Android is a mobile operating system developed...
1,Android has been the best-selling OS worldwide...
2,Google and Twitter made a deal in 2015 that ga...
3,Twitter is an online news and social networkin...
4,"Before entering politics, Donald Trump was a d..."
5,The 2018 FIFA World Cup is the 21st FIFA World...
6,This is the first World Cup to be held in East...
7,Show me the cheapest round trips from Dallas t...
8,I would like to see flights from Denver to Phi...
9,Show me the price of the flights leaving Atlan...


In [162]:
tokenized_test_sentences = []
for x in Test_sentences['message'].values.tolist():
    tokenized_test_sentences.append(word_tokenize(x))

In [163]:
tokenized_test_sentences

[['Android',
  'is',
  'a',
  'mobile',
  'operating',
  'system',
  'developed',
  'by',
  'Google',
  '.'],
 ['Android',
  'has',
  'been',
  'the',
  'best-selling',
  'OS',
  'worldwide',
  'on',
  'smartphones',
  'since',
  '2011',
  'and',
  'on',
  'tablets',
  'since',
  '2013',
  '.'],
 ['Google',
  'and',
  'Twitter',
  'made',
  'a',
  'deal',
  'in',
  '2015',
  'that',
  'gave',
  'Google',
  'access',
  'to',
  'Twitter',
  "'s",
  'firehose',
  '.'],
 ['Twitter',
  'is',
  'an',
  'online',
  'news',
  'and',
  'social',
  'networking',
  'service',
  'on',
  'which',
  'users',
  'post',
  'and',
  'interact',
  'with',
  'messages',
  'known',
  'as',
  'tweets',
  '.'],
 ['Before',
  'entering',
  'politics',
  ',',
  'Donald',
  'Trump',
  'was',
  'a',
  'domineering',
  'businessman',
  'and',
  'a',
  'television',
  'personality',
  '.'],
 ['The',
  '2018',
  'FIFA',
  'World',
  'Cup',
  'is',
  'the',
  '21st',
  'FIFA',
  'World',
  'Cup',
  ',',
  'an',
  'i

In [164]:
test_tagged_words = [word for sentence in tokenized_test_sentences for word in sentence]
test_tagged_words
print(len(test_tagged_words))

181


In [165]:
test_tagged_seq,diff2 = cal_time(test_tagged_words)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


In [166]:
print(test_tagged_seq)

[('Android', 'PRT'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'PRT'), ('.', '.'), ('Android', 'PRT'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'PRT'), ('worldwide', 'PRT'), ('on', 'ADP'), ('smartphones', 'PRT'), ('since', 'ADP'), ('2011', 'PRT'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'PRT'), ('.', '.'), ('Google', 'PRT'), ('and', 'CONJ'), ('Twitter', 'PRT'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'PRT'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'PRT'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'PRT'), ("'s", 'VERB'), ('firehose', 'PRT'), ('.', '.'), ('Twitter', 'PRT'), ('is', 'VERB'), ('an', 'DET'), ('online', 'PRT'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NO

In [167]:
print("Time taken in seconds: ",diff2)

Time taken in seconds:  31.40661597251892


In [168]:
#Custom function which based on "universal" tagset uses the nltk's pos_tag function to output list of POS tagged sentences
def posTag(sentence_list):
    posTags = []    
    for sentence in sentence_list:
        tagged_words = nltk.pos_tag([word for word in word_tokenize(sentence)],tagset='universal')
        posTags.append(tagged_words)
    return posTags

In [169]:
test_run_sentence = posTag(Test_sentences['message'].values.tolist())
test_run_base = [tup for tupsentence in test_run_sentence for tup in tupsentence]

In [170]:
print(test_run_base)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'ADJ'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('

In [171]:
print("Length of test_run_base ", len(test_run_base))

Length of test_run_base  181


In [172]:
print(test_tagged_seq)

[('Android', 'PRT'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'PRT'), ('.', '.'), ('Android', 'PRT'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'PRT'), ('worldwide', 'PRT'), ('on', 'ADP'), ('smartphones', 'PRT'), ('since', 'ADP'), ('2011', 'PRT'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'PRT'), ('.', '.'), ('Google', 'PRT'), ('and', 'CONJ'), ('Twitter', 'PRT'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'PRT'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'PRT'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'PRT'), ("'s", 'VERB'), ('firehose', 'PRT'), ('.', '.'), ('Twitter', 'PRT'), ('is', 'VERB'), ('an', 'DET'), ('online', 'PRT'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NO

In [173]:
print("Length of test_tagged_seq ", len(test_tagged_seq))

Length of test_tagged_seq  181


In [174]:
# Calculating Accuracy

check = [i for i, j in zip(test_tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(test_tagged_seq)

print(accuracy)

0.7569060773480663


### Accuracy is 75.6%

### Incorrectly tagged cases in Test Set 

In [175]:
incorrectly_tagged_cases_test = [[test_run_base[i-1],j] for i, j in enumerate(zip(test_tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrectly_tagged_cases_test

[[('.', '.'), (('Android', 'PRT'), ('Android', 'NOUN'))],
 [('by', 'ADP'), (('Google', 'PRT'), ('Google', 'NOUN'))],
 [('.', '.'), (('Android', 'PRT'), ('Android', 'NOUN'))],
 [('best-selling', 'ADJ'), (('OS', 'PRT'), ('OS', 'NOUN'))],
 [('OS', 'NOUN'), (('worldwide', 'PRT'), ('worldwide', 'NOUN'))],
 [('on', 'ADP'), (('smartphones', 'PRT'), ('smartphones', 'NOUN'))],
 [('since', 'ADP'), (('2011', 'PRT'), ('2011', 'NUM'))],
 [('since', 'ADP'), (('2013', 'PRT'), ('2013', 'NUM'))],
 [('.', '.'), (('Google', 'PRT'), ('Google', 'NOUN'))],
 [('and', 'CONJ'), (('Twitter', 'PRT'), ('Twitter', 'NOUN'))],
 [('in', 'ADP'), (('2015', 'PRT'), ('2015', 'NUM'))],
 [('gave', 'VERB'), (('Google', 'PRT'), ('Google', 'NOUN'))],
 [('to', 'PRT'), (('Twitter', 'PRT'), ('Twitter', 'NOUN'))],
 [('Twitter', 'NOUN'), (("'s", 'VERB'), ("'s", 'PRT'))],
 [("'s", 'PRT'), (('firehose', 'PRT'), ('firehose', 'NOUN'))],
 [('.', '.'), (('Twitter', 'PRT'), ('Twitter', 'NOUN'))],
 [('an', 'DET'), (('online', 'PRT'), ('on

<b>Number of Unique POS tags

In [176]:
tags = [p[1] for p in train_tagged_words]
unique_tags = set(tags)
len(unique_tags)

12

<b>Frequency Counting

In [177]:
from collections import Counter
tag_counts = Counter(tags)
tag_counts.most_common(1)

[('NOUN', 27319)]

Noun is the most occuring tag. We replace all unknown words by Noun.

Replace Tags of unknown words with most common tag(Noun) 

In [195]:
def Viterbi_1(words, bag = train_tagged_words):
    tagged_seq = Viterbi(words)
    T = list(set([p[0] for p in bag]))
    
    words1 = [p[0] for p in tagged_seq]
    Viterbi_tags = [p[1] for p in tagged_seq]
    
    for key, word in enumerate(words1):
        if word not in T:
            Viterbi_tags[key] = 'NOUN'
            
    
    return list(zip(words1, Viterbi_tags))  

In [196]:
Viterbi_valid_set = Viterbi_1(validation_tagged_words)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


In [197]:
# accuracy on validation set
check_1_validation_set = [i for i, j in zip(Viterbi_valid_set, validation_run_base) if i == j] 
accuracy = len(check_1_validation_set)/len(Viterbi_valid_set)

accuracy

0.9016492357200322

### Accuracy is 93.6% for validation set

In [198]:
# Running the model on the test set and checking time
start = time.time()
test_tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


In [199]:
print(test_run_base)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'ADJ'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('

In [200]:
print(len(test_run_base))

181


In [201]:
print(test_tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'VERB'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'

In [202]:
print(len(test_tagged_seq))

181


In [203]:
# accuracy on test set with modified viterbi - 1

check_1_test = [i for i, j in zip(test_tagged_seq, test_run_base) if i == j] 
accuracy_test1 = len(check_1_test)/len(test_tagged_seq)

print(accuracy_test1)

0.9060773480662984


In [204]:
incorrectly_tagged_cases_test_latest = [[test_run_base[i-1],j] for i, j in enumerate(zip(test_tagged_seq, test_run_base)) if j[0]!=j[1]]
print('Incorrectly tagged cases with modified Viterbi - 1')
incorrectly_tagged_cases_test_latest


Incorrectly tagged cases with modified Viterbi - 1


[[('since', 'ADP'), (('2011', 'NOUN'), ('2011', 'NUM'))],
 [('since', 'ADP'), (('2013', 'NOUN'), ('2013', 'NUM'))],
 [('in', 'ADP'), (('2015', 'NOUN'), ('2015', 'NUM'))],
 [('Twitter', 'NOUN'), (("'s", 'VERB'), ("'s", 'PRT'))],
 [('an', 'DET'), (('online', 'NOUN'), ('online', 'ADJ'))],
 [('a', 'DET'), (('domineering', 'NOUN'), ('domineering', 'ADJ'))],
 [('The', 'DET'), (('2018', 'NOUN'), ('2018', 'NUM'))],
 [('the', 'DET'), (('21st', 'NOUN'), ('21st', 'NUM'))],
 [('tournament', 'NOUN'), (('contested', 'NOUN'), ('contested', 'VERB'))],
 [('the', 'DET'), (('11th', 'ADJ'), ('11th', 'NUM'))],
 [('.', '.'), (('Show', 'NOUN'), ('Show', 'VERB'))],
 [('to', 'PRT'), (('Atlanta', 'NOUN'), ('Atlanta', 'VERB'))],
 [('would', 'VERB'), (('like', 'ADP'), ('like', 'VERB'))],
 [('.', '.'), (('Show', 'NOUN'), ('Show', 'VERB'))],
 [('at', 'ADP'), (('about', 'ADP'), ('about', 'ADV'))],
 [('NASA', 'NOUN'), (('invited', 'NOUN'), ('invited', 'VERB'))],
 [('to', 'PRT'), (('experience', 'NOUN'), ('experience'

In [205]:
print('Incorrectly tagged cases with Vanila Viterbi')
incorrectly_tagged_cases_test

Incorrectly tagged cases with Vanila Viterbi


[[('.', '.'), (('Android', 'PRT'), ('Android', 'NOUN'))],
 [('by', 'ADP'), (('Google', 'PRT'), ('Google', 'NOUN'))],
 [('.', '.'), (('Android', 'PRT'), ('Android', 'NOUN'))],
 [('best-selling', 'ADJ'), (('OS', 'PRT'), ('OS', 'NOUN'))],
 [('OS', 'NOUN'), (('worldwide', 'PRT'), ('worldwide', 'NOUN'))],
 [('on', 'ADP'), (('smartphones', 'PRT'), ('smartphones', 'NOUN'))],
 [('since', 'ADP'), (('2011', 'PRT'), ('2011', 'NUM'))],
 [('since', 'ADP'), (('2013', 'PRT'), ('2013', 'NUM'))],
 [('.', '.'), (('Google', 'PRT'), ('Google', 'NOUN'))],
 [('and', 'CONJ'), (('Twitter', 'PRT'), ('Twitter', 'NOUN'))],
 [('in', 'ADP'), (('2015', 'PRT'), ('2015', 'NUM'))],
 [('gave', 'VERB'), (('Google', 'PRT'), ('Google', 'NOUN'))],
 [('to', 'PRT'), (('Twitter', 'PRT'), ('Twitter', 'NOUN'))],
 [('Twitter', 'NOUN'), (("'s", 'VERB'), ("'s", 'PRT'))],
 [("'s", 'PRT'), (('firehose', 'PRT'), ('firehose', 'NOUN'))],
 [('.', '.'), (('Twitter', 'PRT'), ('Twitter', 'NOUN'))],
 [('an', 'DET'), (('online', 'PRT'), ('on

#### <b>The Accuracy on the validation set increased from 90.3% to 93.6%
 <b>The Accuracy on test set increased from 75.6% to 90.6%